In [1]:
import numpy as np
import os
import datetime as datetime
import pandas as pd


# os.system('source /home/jackalak/heartbeat/cdf38_0-dist/bin/definitions.B')
# os.environ["CDF_LIB"] = '/home/jackalak/heartbeat/cdf38_0-dist/lib'
# from spacepy import pycdf
# from nasaomnireader import omnireader

import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from datetime import timedelta


import ftplib
import os


import numpy as np
import os
import datetime as datetime
import pandas as pd





import pickle
import glob



import tensorflow as tf
from tensorflow import keras


import datetime

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow

import numpy as np
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from sklearn import preprocessing


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

import matplotlib.pyplot as plt
%matplotlib notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Input

In [2]:

#https://github.com/tensorflow/tensorflow/issues/956
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import InputSpec 
from tensorflow.python.keras.utils import conv_utils

class PeriodicPadding2D(layers.Layer):

  def __init__(self, padding=1, **kwargs):
    super(PeriodicPadding2D, self).__init__(**kwargs)
    self.padding = conv_utils.normalize_tuple(padding, 1, 'padding')
    self.input_spec = InputSpec(ndim=4)

  def wrap_pad(self, input, size):
    M1 = tf.concat([input[:,:, -size:], input, input[:,:, 0:size]], 2)
    M1 = tf.concat([M1[:,0:size, :], M1, M1[:,0:size, :]], 1) #not periodic
    return M1

  def compute_output_shape(self, input_shape):
    shape = list(input_shape)
    assert len(shape) == 3  
    if shape[1] is not None:
      length = shape[1] + 2*self.padding[0]
    else:
      length = None
    return tuple([shape[0], length, length])

  def call(self, inputs): 
    return self.wrap_pad(inputs, self.padding[0])

  def get_config(self):
    config = {'padding': self.padding}
    base_config = super(PeriodicPadding2D, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))


In [3]:
def plot_model(df_results, test):
    
    subdir = 'figures2/energyflux/'

    y_val_log = np.log10(test['ELE_TOTAL_ENERGY_FLUX'])
    
    plt.figure()
    plt.title('Electron Total Energy Flux, log10 scale')    
    plt.plot(y_val_log[:],alpha=0.5)
    plt.plot(df_results[:],alpha=0.5)
    plt.legend(['val', 'result'], loc='upper left')
    plt.ylabel('log10 eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'1',dpi=dpi)
    
    plt.figure()
    plt.title('Electron Total Energy Flux')   
    plt.plot(10**y_val_log[:]*1.6e-6,alpha=0.5)
    plt.plot(10**df_results[:]*1.6e-6,alpha=0.5)
    plt.legend(['val', 'result'], loc='upper left')
    plt.ylabel('erg/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'2',dpi=dpi)
    start=int(y_val_log.shape[0]/2)
    plt.figure()
    plt.title('Electron Total Energy Flux, log10 scale')   
    plt.plot(y_val_log[start:2000+start],alpha=0.5)
    plt.plot(df_results[start:2000+start],alpha=0.5)
    plt.legend(['val', 'result'], loc='upper left')
    plt.ylabel('log10 eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'3',dpi=dpi)
    plt.figure()
    plt.title('Electron Total Energy Flux')   
    plt.plot(10**y_val_log[start:2000+start]*1.6e-6,alpha=0.5)
    plt.plot(10**df_results[start:2000+start]*1.6e-6,alpha=0.5)
    plt.legend(['val', 'result'], loc='upper left')
    plt.ylabel('erg/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'4',dpi=dpi)
    

    minr = np.min(y_val_log.values)
    maxr = np.max(y_val_log.values)
    plt.figure()
    plt.title('Histogram of Electron Total Energy Flux, log10 scale')   
    plt.hist(y_val_log.values,bins=200,alpha=0.5,range=(minr,maxr))
    plt.hist(df_results.values,bins=200,alpha=0.5,range=(minr,maxr))
    plt.legend(['val','result'], loc='upper left')
    plt.ylabel('#/bin')
    plt.xlabel('log10 eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'5',dpi=dpi)
    plt.figure()
    plt.title('Histogram of Electron Total Energy Flux')   
    plt.hist(10**y_val_log.values*1.6e-6,bins=100, log=True,range=(10**minr*1.6e-6,10**maxr*1.6e-6),alpha=0.5)
    plt.hist(10**df_results.values*1.6e-6,bins=100, log=True,range=(10**minr*1.6e-6,10**maxr*1.6e-6),alpha=0.5)
    plt.legend(['val', 'result'], loc='upper left')
    plt.ylabel('#/bin')
    plt.xlabel('erg/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'6',dpi=dpi)
    import matplotlib.colors as mcolors
    gamma = 0.2#[0.8, 0.5, 0.3]

    errors= y_val_log.values-df_results.values[:,0]
    plt.figure();
    plt.hist2d(test['SC_AACGM_LAT'].values, errors,
                  bins=50, norm=mcolors.PowerNorm(gamma))
    plt.title('Error density over SC_AACGM_LAT Bins')
    plt.xlabel('SC_AACGM_LAT degrees')
    plt.ylabel('log10(y_true)-log10(y_pred) eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'7',dpi=dpi)
    plt.figure()
    bin_total = np.zeros((200))
    bin_error_total = np.zeros((200))
    for j in range(0,y_val_log.values.shape[0]):
        i = int((test['SC_AACGM_LAT'].values[j]-45)/((90-45)/200))
        if i < 200:
            bin_total[i] = bin_total[i]+1
            bin_error_total[i] = bin_error_total[i] + np.abs(errors[j])

    avg_error_over_hist = bin_error_total/bin_total
    plt.scatter(np.linspace(45,90,num=200),avg_error_over_hist)
    plt.title('Average Validation Error over SC_AACGM_LAT Bins')
    plt.xlabel('SC_AACGM_LAT degrees')
    plt.ylabel('log10(y_true)-log10(y_pred) eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'8',dpi=dpi)
    bin_total = np.zeros((200))
    bin_error_total = np.zeros((200))
    for j in range(0,y_val_log.values.shape[0]):
        i = int((y_val_log[j]-minr)/((maxr-minr)/200))
        if i < 200:
            bin_total[i] = bin_total[i]+1
            bin_error_total[i] = bin_error_total[i] + np.abs(errors[j])

    avg_error_over_hist = bin_error_total/(bin_total+.00001)
    plt.figure()
    plt.scatter(np.linspace(minr,maxr,num=200),avg_error_over_hist)
    plt.title('Average Validation Error over target Bins')    
    plt.xlabel('log10(y_true) eV/cm^2/ster/s')
    plt.ylabel('log10(y_true)-log10(y_pred) eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'9',dpi=dpi)
    import matplotlib.colors as mcolors
    gamma = 0.2#[0.8, 0.5, 0.3]
    errors= y_val_log.values-df_results.values[:,0]

    plt.figure();
    plt.hist2d(y_val_log.values, errors,
                  bins=50, norm=mcolors.PowerNorm(gamma))
    plt.colorbar()
    plt.title('Error Density')
    plt.xlabel('log10(y_true) eV/cm^2/ster/s')
    plt.ylabel('log10(y_true)-log10(y_pred) eV/cm^2/ster/s')
    plt.show()
    plt.savefig(subdir+'10',dpi=dpi)
    plt.figure();
    plt.hist2d(y_val_log.values, df_results.values[:,0],
                  bins=50, norm=mcolors.PowerNorm(gamma))
    plt.colorbar()
    plt.title('Error Density')
    plt.xlabel('log10(y_true) eV/cm^2/ster/s')
    plt.ylabel('log10(y_pred) eV/cm^2/ster/s')
    temp = np.array([7,8,9,10,11,12,13])
    plt.plot(temp,temp,color='k')
    plt.xlim([7,13])
    plt.ylim([7,13])
    plt.show()
    plt.savefig(subdir+'11',dpi=dpi)

In [4]:

# # LOAD THE COMBINED SOURTHERN HEMISPHERE AND NORTHERN HEMISPHERE AS CREATED ABOVE
# test = pd.DataFrame()

# for ii in [17,18,6,7,8,9,12,13,14,15,16,]:

#     filescname = "../Downloads/sc_df_cleaned_GFi_" + str(ii) + ".pkl"
#     pickle_file = open(filescname, "rb") 

#     test =pd.concat([test,pickle.load(pickle_file)] )
    
# # filescname = "./all_sc_df_1min_SH_NH_combined.pkl"
# # pickle_file = open(filescname, "rb") 
# # test =pickle.load(pickle_file)
# print(test.columns.tolist())

In [5]:

# LOAD THE COMBINED SOURTHERN HEMISPHERE AND NORTHERN HEMISPHERE AS CREATED ABOVE
df_cumulative = pd.DataFrame()
# for ii in [17,18,6,7,8,9,12,13,14,15,16,]:
#     filescname = "./sc_df_cleaned_GFi_" + str(ii) + ".pkl"
#     pickle_file = open(filescname, "rb") 
#     df_cumulative =pd.concat([df_cumulative,pickle.load(pickle_file)] )

filescname = "/home/jackalak/heartbeat/ParticlePrecipitation/ML_DB_subsamp_ext_full_dfCumulative_complexHemisphereCombine.csv"
file= open(filescname, "rb") 
df_cumulative =pd.read_csv(file)
df_cumulative = df_cumulative.set_index('Datetimes')
df_cumulative.index = pd.to_datetime(df_cumulative.index)
print(df_cumulative.columns.tolist())

df_cumulative=df_cumulative[['SC_AACGM_LAT','SC_AACGM_LTIME', 'ELE_TOTAL_ENERGY_FLUX', 'ID_SC', 'sin_ut',
                             'cos_ut', 'sin_doy', 'cos_doy', 'sin_SC_AACGM_LTIME', 'cos_SC_AACGM_LTIME',
                               'F107', 'Bz', 'By', 'Bx', 'AE', 'AL', 'AU', 'SymH', 'PC', 'vsw', 'vx', 'psw', 'borovsky', 'newell', 'F107_6hr', 'Bz_6hr', 'By_6hr', 'Bx_6hr', 'AE_6hr', 'AL_6hr', 'AU_6hr', 'SymH_6hr', 'PC_6hr', 'vsw_6hr', 'vx_6hr', 'psw_6hr', 'borovsky_6hr', 'newell_6hr', 'F107_5hr', 'Bz_5hr', 'By_5hr', 'Bx_5hr', 'AE_5hr', 'AL_5hr', 'AU_5hr', 'SymH_5hr', 'PC_5hr', 'vsw_5hr', 'vx_5hr', 'psw_5hr', 'borovsky_5hr', 'newell_5hr', 'F107_3hr', 'Bz_3hr', 'By_3hr', 'Bx_3hr', 'AE_3hr', 'AL_3hr', 'AU_3hr', 'SymH_3hr', 'PC_3hr', 'vsw_3hr', 'vx_3hr', 'psw_3hr', 'borovsky_3hr', 'newell_3hr', 'F107_1hr', 'Bz_1hr', 'By_1hr', 'Bx_1hr', 'AE_1hr', 'AL_1hr', 'AU_1hr', 'SymH_1hr', 'PC_1hr', 'vsw_1hr', 'vx_1hr', 'psw_1hr', 'borovsky_1hr', 'newell_1hr', 'F107_45min', 'Bz_45min', 'By_45min', 'Bx_45min', 'AE_45min', 'AL_45min', 'AU_45min', 'SymH_45min', 'PC_45min', 'vsw_45min', 'vx_45min', 'psw_45min', 'borovsky_45min', 'newell_45min', 'F107_30min', 'Bz_30min', 'By_30min', 'Bx_30min', 'AE_30min', 'AL_30min', 'AU_30min', 'SymH_30min', 'PC_30min', 'vsw_30min', 'vx_30min', 'psw_30min', 'borovsky_30min', 'newell_30min', 'F107_15min', 'Bz_15min', 'By_15min', 'Bx_15min', 'AE_15min', 'AL_15min', 'AU_15min', 'SymH_15min', 'PC_15min', 'vsw_15min', 'vx_15min', 'psw_15min', 'borovsky_15min', 'newell_15min', 'F107_10min', 'Bz_10min', 'By_10min', 'Bx_10min', 'AE_10min', 'AL_10min', 'AU_10min', 'SymH_10min', 'PC_10min', 'vsw_10min', 'vx_10min', 'psw_10min', 'borovsky_10min', 'newell_10min', 'F107_5min', 'Bz_5min', 'By_5min', 'Bx_5min', 'AE_5min', 'AL_5min', 'AU_5min', 'SymH_5min', 'PC_5min', 'vsw_5min', 'vx_5min', 'psw_5min', 'borovsky_5min', 'newell_5min'        ]]

df_cumulative.sort_index()
df_cumulative

# Separate training and testing data
mask_val = [(df_cumulative.index.year == 2010) & (df_cumulative['ID_SC'].values==16)]
df_val = df_cumulative[mask_val[0]].copy(deep=True)
df_train = df_cumulative.copy(deep=True).drop( df_cumulative.index[mask_val[0]])

del df_cumulative

feature_cols = [ 'ID_SC', 'sin_ut','cos_ut', 'sin_doy', 'cos_doy', 
                     'F107', 'Bz', 'By', 'Bx', 'AE', 'AL', 'AU', 'SymH', 'PC', 'vsw', 'vx', 'psw', 'borovsky', 'newell', 'F107_6hr', 'Bz_6hr', 'By_6hr', 'Bx_6hr', 'AE_6hr', 'AL_6hr', 'AU_6hr', 'SymH_6hr', 'PC_6hr', 'vsw_6hr', 'vx_6hr', 'psw_6hr', 'borovsky_6hr', 'newell_6hr', 'F107_5hr', 'Bz_5hr', 'By_5hr', 'Bx_5hr', 'AE_5hr', 'AL_5hr', 'AU_5hr', 'SymH_5hr', 'PC_5hr', 'vsw_5hr', 'vx_5hr', 'psw_5hr', 'borovsky_5hr', 'newell_5hr', 'F107_3hr', 'Bz_3hr', 'By_3hr', 'Bx_3hr', 'AE_3hr', 'AL_3hr', 'AU_3hr', 'SymH_3hr', 'PC_3hr', 'vsw_3hr', 'vx_3hr', 'psw_3hr', 'borovsky_3hr', 'newell_3hr', 'F107_1hr', 'Bz_1hr', 'By_1hr', 'Bx_1hr', 'AE_1hr', 'AL_1hr', 'AU_1hr', 'SymH_1hr', 'PC_1hr', 'vsw_1hr', 'vx_1hr', 'psw_1hr', 'borovsky_1hr', 'newell_1hr', 'F107_45min', 'Bz_45min', 'By_45min', 'Bx_45min', 'AE_45min', 'AL_45min', 'AU_45min', 'SymH_45min', 'PC_45min', 'vsw_45min', 'vx_45min', 'psw_45min', 'borovsky_45min', 'newell_45min', 'F107_30min', 'Bz_30min', 'By_30min', 'Bx_30min', 'AE_30min', 'AL_30min', 'AU_30min', 'SymH_30min', 'PC_30min', 'vsw_30min', 'vx_30min', 'psw_30min', 'borovsky_30min', 'newell_30min', 'F107_15min', 'Bz_15min', 'By_15min', 'Bx_15min', 'AE_15min', 'AL_15min', 'AU_15min', 'SymH_15min', 'PC_15min', 'vsw_15min', 'vx_15min', 'psw_15min', 'borovsky_15min', 'newell_15min', 'F107_10min', 'Bz_10min', 'By_10min', 'Bx_10min', 'AE_10min', 'AL_10min', 'AU_10min', 'SymH_10min', 'PC_10min', 'vsw_10min', 'vx_10min', 'psw_10min', 'borovsky_10min', 'newell_10min', 'F107_5min', 'Bz_5min', 'By_5min', 'Bx_5min', 'AE_5min', 'AL_5min', 'AU_5min', 'SymH_5min', 'PC_5min', 'vsw_5min', 'vx_5min', 'psw_5min', 'borovsky_5min', 'newell_5min'               ]
print(len(feature_cols))

# Construct X and y
# feature_cols = [c for c in df_train.columns if not 'ELE' in c]
# print( (feature_cols))
#print(df_cumulative.columns)
X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))
XX = np.array(X_train_scaled, dtype=np.float32)
XX_test = np.array(X_val_scaled, dtype=np.float32)

del X_val, X_train

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input

['SC_AACGM_LAT', 'SC_AACGM_LTIME', 'ELE_TOTAL_ENERGY_FLUX', 'ELE_TOTAL_ENERGY_FLUX_STD', 'ELE_AVG_ENERGY', 'ELE_AVG_ENERGY_STD', 'ID_SC', 'sin_ut', 'cos_ut', 'sin_doy', 'cos_doy', 'sin_SC_AACGM_LTIME', 'cos_SC_AACGM_LTIME', 'F107', 'Bz', 'By', 'Bx', 'AE', 'AL', 'AU', 'SymH', 'PC', 'vsw', 'vx', 'psw', 'borovsky', 'newell', 'F107_6hr', 'Bz_6hr', 'By_6hr', 'Bx_6hr', 'AE_6hr', 'AL_6hr', 'AU_6hr', 'SymH_6hr', 'PC_6hr', 'vsw_6hr', 'vx_6hr', 'psw_6hr', 'borovsky_6hr', 'newell_6hr', 'F107_5hr', 'Bz_5hr', 'By_5hr', 'Bx_5hr', 'AE_5hr', 'AL_5hr', 'AU_5hr', 'SymH_5hr', 'PC_5hr', 'vsw_5hr', 'vx_5hr', 'psw_5hr', 'borovsky_5hr', 'newell_5hr', 'F107_3hr', 'Bz_3hr', 'By_3hr', 'Bx_3hr', 'AE_3hr', 'AL_3hr', 'AU_3hr', 'SymH_3hr', 'PC_3hr', 'vsw_3hr', 'vx_3hr', 'psw_3hr', 'borovsky_3hr', 'newell_3hr', 'F107_1hr', 'Bz_1hr', 'By_1hr', 'Bx_1hr', 'AE_1hr', 'AL_1hr', 'AU_1hr', 'SymH_1hr', 'PC_1hr', 'vsw_1hr', 'vx_1hr', 'psw_1hr', 'borovsky_1hr', 'newell_1hr', 'F107_45min', 'Bz_45min', 'By_45min', 'Bx_45min', 'A

In [6]:
df_train

,SC_AACGM_LAT,SC_AACGM_LTIME,ELE_TOTAL_ENERGY_FLUX,ID_SC,sin_ut,cos_ut,sin_doy,cos_doy,sin_SC_AACGM_LTIME,cos_SC_AACGM_LTIME,...,AE_5min,AL_5min,AU_5min,SymH_5min,PC_5min,vsw_5min,vx_5min,psw_5min,borovsky_5min,newell_5min
Datetimes,,,,,,,,,,,,,,,,,,,,,
1987-01-12 12:57:00,51.143220,5.664024,2.057827e+08,6.0,-0.246153,-0.969231,2.051045e-01,0.97874,0.996134,0.087845,...,86.0,-61.0,25.0,3.0,0.77,356.0,-355.5,2.96,17439.640589,2489.686963
1987-01-12 12:58:00,54.137207,5.535848,2.509837e+08,6.0,-0.250380,-0.968148,2.051045e-01,0.97874,0.992626,0.121216,...,86.0,-61.0,25.0,3.0,0.77,356.0,-355.5,2.96,17439.640589,2489.686963
1987-01-12 13:01:00,62.928176,5.031172,4.469146e+08,6.0,-0.263031,-0.964787,2.051045e-01,0.97874,0.968006,0.250928,...,82.0,-58.0,24.0,3.0,0.77,354.7,-354.1,3.20,14199.368957,1847.748015
1987-01-12 13:02:00,65.719631,4.794780,1.934849e+10,6.0,-0.267238,-0.963630,2.051045e-01,0.97874,0.950633,0.310316,...,82.0,-58.0,24.0,3.0,0.77,354.7,-354.1,3.20,14199.368957,1847.748015
1987-01-12 13:03:00,68.392364,4.507448,3.913337e+11,6.0,-0.271440,-0.962455,2.051045e-01,0.97874,0.924624,0.380881,...,82.0,-58.0,24.0,3.0,0.77,354.7,-354.1,3.20,14199.368957,1847.748015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 22:50:00,57.581441,19.667482,2.231577e+08,18.0,-0.300706,0.953717,6.432491e-16,1.00000,-0.906218,0.422812,...,52.0,-14.0,38.0,-11.0,-0.12,528.8,-528.3,2.14,8055.887423,622.199021
2014-12-31 22:51:00,54.923541,19.585868,3.074197e+08,18.0,-0.296542,0.955020,6.432491e-16,1.00000,-0.915044,0.403354,...,52.0,-14.0,38.0,-11.0,-0.12,528.8,-528.3,2.14,8055.887423,622.199021
2014-12-31 22:52:00,52.270565,19.512302,1.917582e+08,18.0,-0.292372,0.956305,6.432491e-16,1.00000,-0.922642,0.385657,...,52.0,-14.0,38.0,-11.0,-0.12,528.8,-528.3,2.14,8055.887423,622.199021


In [7]:
import numpy as np

class DataGenerator_train(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(33), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        print(list_IDs)
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim))
        y = np.zeros((self.batch_size,128,128))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] =XX[ID,:] #np.load('data/' + ID + '.npy')
            for j in range(0,15): # will use moving window for 5 timesteps up to 3 SC_IDs
                if ((ID-7+j > 0) and (ID-7+j)<len(self.list_IDs)):
                    if(abs( (df_train.index[ID]-
                       df_train.index[ID-7+j] ).seconds) <=2*60 ):
                        
                        mlat = df_train['SC_AACGM_LAT'][ID-7+j]
                        mltime = df_train['SC_AACGM_LTIME'][ID-7+j]/24*2*np.pi 
                        mlat_j = int((90-mlat)/45*128)
                        mltime_k = int((mltime)/(2*np.pi)*128)

                        y[i,mlat_j,mltime_k] = y_train_log[ID-7+j]

        return X, y
    

class DataGenerator_test(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(33), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        print(list_IDs)
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.dim))
        y = np.zeros((self.batch_size,128,128))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] =XX_test[ID,:] #np.load('data/' + ID + '.npy')
            mlat = df_val['SC_AACGM_LAT'][ID]
            mltime = df_val['SC_AACGM_LTIME'][ID]/24*2*np.pi 
            mlat_j = int((90-mlat)/45*128)
            mltime_k = int((mltime)/(2*np.pi)*128)
            #print( mlat,   mltime ,mlat_j,  mltime_k )


            y[i,mlat_j,mltime_k] = y_val_log[ID]

        return X, y
 

In [8]:
num_in = 145

# Parameters
params = {'dim': 145,
          'batch_size': 4096,
          'n_classes': 6, # not used
          'n_channels': 1, # not used
          'shuffle': True}

# Datasets
partition ={ 'train': np.linspace(0, df_train.shape[0]-1, df_train.shape[0]).astype(int), 'validation': np.linspace(0, df_val.shape[0]-1, df_val.shape[0]).astype(int)}
labels = { 'train': np.linspace(0, df_train.shape[0]-1, df_train.shape[0]).astype(int), 'validation': np.linspace(0, df_val.shape[0]-1, df_val.shape[0]).astype(int)}

# Generators
training_generator = DataGenerator_train(partition['train'], labels, **params)
validation_generator = DataGenerator_test(partition['validation'], labels, **params)

def custom_loss(y_true, y_pred):
  
    loss = K.sum( K.cast(K.greater(y_true, 0),'float32')*
               K.square(y_true-y_pred) )  / ( 
                K.sum( K.cast(K.greater(y_true, 0),'float32') ))# finds the number of y_true  > 0
    return loss



def custom_mse(y_true, y_pred):
    

    
    mse = K.sum( K.cast(K.greater(y_true, 0),'float32')*(
        K.square(y_true-y_pred)) )/params['batch_size']  
    return mse



[      0       1       2 ... 1838280 1838281 1838282]
[    0     1     2 ... 55207 55208 55209]


In [9]:
input1 = Input(shape=(num_in))

model1 = Dense(256, activation='relu')(input1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(int(64),   activation='relu')(model1)
model1 = Dense(int(32),   activation='relu')(model1)
model1 = Dense(int(256),   activation='relu')(model1)
model1 = tf.keras.layers.Reshape((16, 16, 1))(model1)

# 16x16 to 16 32x32 feature map
model1 = tf.keras.layers.Conv2DTranspose(4, (9,9),
                strides=(2,2), padding='same')(model1)
model1 = tf.keras.layers.Conv2DTranspose(4, (5,5),
                strides=(4,4), padding='same')(model1)
# 16 1286x128 feature map to 1 128x128
model1 = model1 = Dropout(0.5)(model1)
model1 = PeriodicPadding2D(3)(model1)
model1 = tf.keras.layers.Conv2D(1, kernel_size=(7,7),
                              padding='valid')(model1)
#add dropout or max pooling?

output = model1

model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss=custom_loss, optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001),metrics=custom_mse)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 145)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               37376     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               8448      
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 1)        

In [10]:
history = model.fit(training_generator, 
                    validation_data=validation_generator,
                    batch_size=params['batch_size'],epochs=300,#verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=60)], use_multiprocessing=True,
                     workers=6)#

Epoch 1/300
448/448 [==============================] - 343s 765ms/step - loss: 12.2439 - custom_mse: 32.4797 - val_loss: 1.0314 - val_custom_mse: 1.0314
Epoch 2/300
448/448 [==============================] - 342s 764ms/step - loss: 1.1844 - custom_mse: 3.1423 - val_loss: 0.9174 - val_custom_mse: 0.9174
Epoch 3/300
448/448 [==============================] - 346s 773ms/step - loss: 1.0258 - custom_mse: 2.7216 - val_loss: 0.8795 - val_custom_mse: 0.8795
Epoch 4/300
448/448 [==============================] - 355s 792ms/step - loss: 0.9725 - custom_mse: 2.5801 - val_loss: 0.8518 - val_custom_mse: 0.8518
Epoch 5/300
448/448 [==============================] - 368s 821ms/step - loss: 0.9435 - custom_mse: 2.5033 - val_loss: 0.8392 - val_custom_mse: 0.8392
Epoch 6/300
448/448 [==============================] - 374s 835ms/step - loss: 0.9265 - custom_mse: 2.4580 - val_loss: 0.8311 - val_custom_mse: 0.8311
Epoch 7/300
448/448 [==============================] - 364s 813ms/step - loss: 0.9133 - cust

Epoch 9/300
448/448 [==============================] - 356s 795ms/step - loss: 0.8910 - custom_mse: 2.3640 - val_loss: 0.7983 - val_custom_mse: 0.7983
Epoch 10/300
448/448 [==============================] - 411s 917ms/step - loss: 0.8804 - custom_mse: 2.3356 - val_loss: 0.7967 - val_custom_mse: 0.7967
Epoch 11/300
448/448 [==============================] - 383s 856ms/step - loss: 0.8671 - custom_mse: 2.3005 - val_loss: 0.7882 - val_custom_mse: 0.7882
Epoch 12/300
448/448 [==============================] - 385s 860ms/step - loss: 0.8528 - custom_mse: 2.2625 - val_loss: 0.7683 - val_custom_mse: 0.7683
Epoch 13/300
448/448 [==============================] - 368s 822ms/step - loss: 0.8374 - custom_mse: 2.2218 - val_loss: 0.7434 - val_custom_mse: 0.7434
Epoch 14/300
448/448 [==============================] - 355s 793ms/step - loss: 0.8192 - custom_mse: 2.1733 - val_loss: 0.7359 - val_custom_mse: 0.7359
Epoch 15/300
448/448 [==============================] - 356s 795ms/step - loss: 0.8005 - 

448/448 [==============================] - 346s 773ms/step - loss: 0.5855 - custom_mse: 1.5535 - val_loss: 0.6242 - val_custom_mse: 0.6242
Epoch 26/300
448/448 [==============================] - 349s 780ms/step - loss: 0.5683 - custom_mse: 1.5077 - val_loss: 0.6226 - val_custom_mse: 0.6226
Epoch 27/300
448/448 [==============================] - 347s 774ms/step - loss: 0.5524 - custom_mse: 1.4656 - val_loss: 0.6177 - val_custom_mse: 0.6177
Epoch 28/300
448/448 [==============================] - 353s 787ms/step - loss: 0.5378 - custom_mse: 1.4270 - val_loss: 0.6175 - val_custom_mse: 0.6175
Epoch 29/300
448/448 [==============================] - 364s 812ms/step - loss: 0.5241 - custom_mse: 1.3906 - val_loss: 0.6112 - val_custom_mse: 0.6112
Epoch 30/300
448/448 [==============================] - 343s 766ms/step - loss: 0.5122 - custom_mse: 1.3590 - val_loss: 0.6114 - val_custom_mse: 0.6114
Epoch 31/300
448/448 [==============================] - 349s 780ms/step - loss: 0.5012 - custom_mse: 

448/448 [==============================] - 350s 782ms/step - loss: 0.4831 - custom_mse: 1.2816 - val_loss: 0.6064 - val_custom_mse: 0.6064
Epoch 34/300
448/448 [==============================] - 346s 773ms/step - loss: 0.4751 - custom_mse: 1.2604 - val_loss: 0.6034 - val_custom_mse: 0.6034
Epoch 35/300
448/448 [==============================] - 349s 779ms/step - loss: 0.4684 - custom_mse: 1.2427 - val_loss: 0.6068 - val_custom_mse: 0.6068
Epoch 36/300
448/448 [==============================] - 352s 785ms/step - loss: 0.4620 - custom_mse: 1.2259 - val_loss: 0.6011 - val_custom_mse: 0.6011
Epoch 37/300
448/448 [==============================] - 353s 789ms/step - loss: 0.4565 - custom_mse: 1.2112 - val_loss: 0.6034 - val_custom_mse: 0.6034
Epoch 38/300
448/448 [==============================] - 353s 788ms/step - loss: 0.4524 - custom_mse: 1.2004 - val_loss: 0.6022 - val_custom_mse: 0.6022
Epoch 39/300
448/448 [==============================] - 344s 769ms/step - loss: 0.4478 - custom_mse: 

448/448 [==============================] - 346s 773ms/step - loss: 0.4392 - custom_mse: 1.1652 - val_loss: 0.6025 - val_custom_mse: 0.6025
Epoch 43/300
448/448 [==============================] - 353s 788ms/step - loss: 0.4368 - custom_mse: 1.1589 - val_loss: 0.5982 - val_custom_mse: 0.5982
Epoch 44/300
448/448 [==============================] - 414s 925ms/step - loss: 0.4349 - custom_mse: 1.1539 - val_loss: 0.5998 - val_custom_mse: 0.5998
Epoch 45/300
448/448 [==============================] - 370s 826ms/step - loss: 0.4335 - custom_mse: 1.1502 - val_loss: 0.6005 - val_custom_mse: 0.6005
Epoch 46/300
448/448 [==============================] - 367s 820ms/step - loss: 0.4322 - custom_mse: 1.1467 - val_loss: 0.5976 - val_custom_mse: 0.5976
Epoch 47/300
448/448 [==============================] - 360s 803ms/step - loss: 0.4309 - custom_mse: 1.1434 - val_loss: 0.6058 - val_custom_mse: 0.6058
Epoch 48/300
448/448 [==============================] - 354s 790ms/step - loss: 0.4301 - custom_mse: 

...

448/448 [==============================] - 366s 818ms/step - loss: 0.4199 - custom_mse: 1.1141 - val_loss: 0.5914 - val_custom_mse: 0.5914
Epoch 84/300
448/448 [==============================] - 368s 822ms/step - loss: 0.4198 - custom_mse: 1.1137 - val_loss: 0.5901 - val_custom_mse: 0.5901
Epoch 85/300
448/448 [==============================] - 363s 810ms/step - loss: 0.4198 - custom_mse: 1.1137 - val_loss: 0.5864 - val_custom_mse: 0.5864
Epoch 86/300
448/448 [==============================] - 362s 808ms/step - loss: 0.4194 - custom_mse: 1.1126 - val_loss: 0.5880 - val_custom_mse: 0.5880
Epoch 87/300
448/448 [==============================] - 358s 798ms/step - loss: 0.4190 - custom_mse: 1.1117 - val_loss: 0.5880 - val_custom_mse: 0.5880
Epoch 88/300
448/448 [==============================] - 349s 778ms/step - loss: 0.4195 - custom_mse: 1.1130 - val_loss: 0.5843 - val_custom_mse: 0.5843
Epoch 89/300
448/448 [==============================] - 355s 792ms/step - loss: 0.4191 - custom_mse: 

448/448 [==============================] - 357s 796ms/step - loss: 0.4187 - custom_mse: 1.1108 - val_loss: 0.5876 - val_custom_mse: 0.5876
Epoch 93/300
448/448 [==============================] - 349s 780ms/step - loss: 0.4186 - custom_mse: 1.1105 - val_loss: 0.5893 - val_custom_mse: 0.5893
Epoch 94/300
448/448 [==============================] - 351s 784ms/step - loss: 0.4184 - custom_mse: 1.1100 - val_loss: 0.5853 - val_custom_mse: 0.5853
Epoch 95/300
448/448 [==============================] - 345s 771ms/step - loss: 0.4185 - custom_mse: 1.1102 - val_loss: 0.5853 - val_custom_mse: 0.5853
Epoch 96/300
448/448 [==============================] - 347s 775ms/step - loss: 0.4180 - custom_mse: 1.1089 - val_loss: 0.5863 - val_custom_mse: 0.5863
Epoch 97/300
448/448 [==============================] - 350s 782ms/step - loss: 0.4181 - custom_mse: 1.1093 - val_loss: 0.5864 - val_custom_mse: 0.5864
Epoch 98/300
448/448 [==============================] - 349s 780ms/step - loss: 0.4183 - custom_mse: 

448/448 [==============================] - 350s 782ms/step - loss: 0.4172 - custom_mse: 1.1069 - val_loss: 0.5805 - val_custom_mse: 0.5805
Epoch 109/300
448/448 [==============================] - 351s 782ms/step - loss: 0.4172 - custom_mse: 1.1070 - val_loss: 0.5830 - val_custom_mse: 0.5830
Epoch 110/300
448/448 [==============================] - 357s 797ms/step - loss: 0.4172 - custom_mse: 1.1070 - val_loss: 0.5824 - val_custom_mse: 0.5824
Epoch 111/300
448/448 [==============================] - 347s 775ms/step - loss: 0.4169 - custom_mse: 1.1060 - val_loss: 0.5929 - val_custom_mse: 0.5929
Epoch 112/300
448/448 [==============================] - 351s 783ms/step - loss: 0.4167 - custom_mse: 1.1056 - val_loss: 0.5863 - val_custom_mse: 0.5863
Epoch 113/300
448/448 [==============================] - 346s 773ms/step - loss: 0.4165 - custom_mse: 1.1050 - val_loss: 0.5843 - val_custom_mse: 0.5843
Epoch 114/300
448/448 [==============================] - 342s 763ms/step - loss: 0.4168 - custom

Epoch 117/300
448/448 [==============================] - 384s 857ms/step - loss: 0.4162 - custom_mse: 1.1041 - val_loss: 0.5842 - val_custom_mse: 0.5842
Epoch 118/300
448/448 [==============================] - 366s 817ms/step - loss: 0.4162 - custom_mse: 1.1042 - val_loss: 0.5859 - val_custom_mse: 0.5859
Epoch 119/300
448/448 [==============================] - 403s 899ms/step - loss: 0.4160 - custom_mse: 1.1038 - val_loss: 0.5862 - val_custom_mse: 0.5862
Epoch 120/300
448/448 [==============================] - 406s 906ms/step - loss: 0.4160 - custom_mse: 1.1037 - val_loss: 0.5888 - val_custom_mse: 0.5888
Epoch 121/300
448/448 [==============================] - 407s 907ms/step - loss: 0.4161 - custom_mse: 1.1040 - val_loss: 0.5886 - val_custom_mse: 0.5886
Epoch 122/300
448/448 [==============================] - 411s 918ms/step - loss: 0.4163 - custom_mse: 1.1046 - val_loss: 0.5830 - val_custom_mse: 0.5830
Epoch 123/300
448/448 [==============================] - 406s 906ms/step - loss: 0

Process Keras_worker_ForkPoolWorker-3170:
Process Keras_worker_ForkPoolWorker-3172:
Process Keras_worker_ForkPoolWorker-3174:
Process Keras_worker_ForkPoolWorker-3168:
Process Keras_worker_ForkPoolWorker-3165:
Process Keras_worker_ForkPoolWorker-3164:
Process Keras_worker_ForkPoolWorker-3171:
Process Keras_worker_ForkPoolWorker-3169:
Process Keras_worker_ForkPoolWorker-3166:
Process Keras_worker_ForkPoolWorker-3167:
Process Keras_worker_ForkPoolWorker-3163:
Process Keras_worker_ForkPoolWorker-3173:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.

In [17]:
filename='mse_new_pipeline_33_2D_128x128_periodic_2_dropout'
model.save(filename)
model = tensorflow.keras.models.load_model(filename, custom_objects={'custom_loss': custom_loss, 'custom_mse': custom_mse})

plt.figure()
# summarize history for loss
plt.plot(history.history['loss'][:])
plt.plot(history.history['val_loss'][:])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.ylim([0,.9])
plt.show()

plt.figure()
# summarize history for loss
plt.plot(history.history['loss'][:])
plt.plot(history.history['val_loss'][:])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



INFO:tensorflow:Assets written to: mse_new_pipeline_33_2D_128x128_periodic_2_dropout/assets


NameError: name 'history' is not defined

In [ ]:
history = model.fit(training_generator, 
                    validation_data=validation_generator,
                    batch_size=params['batch_size'],epochs=300,#verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=60)], use_multiprocessing=True,
                     workers=6)#

Epoch 1/300


Process Keras_worker_ForkPoolWorker-3215:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 341, in put
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
KeyboardInterrupt
Process Keras_worker_ForkPoolWorker-3218:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job

In [13]:
mark= ['s', 'o', 'D', 'v']

%matplotlib inline
plt.ioff()
range1=np.min(np.min([-df_val['AL'].values,df_val['AE'].values,df_val['AU'].values]))
range2=np.max(np.max([-df_val['AL'].values,df_val['AE'].values,df_val['AU'].values]))
os.system('source /home/jackalak/heartbeat/cdf38_0-dist/bin/definitions.B')
os.environ["CDF_LIB"] = '/home/jackalak/heartbeat/cdf38_0-dist/lib'
from spacepy import pycdf

from ovationpyme.ovation_prime import FluxEstimator,AverageEnergyEstimator,BinCorrector
from ovationpyme.ovation_utilities import calc_avg_solarwind
from ovationpyme.ovation_plotting import latlt2polar,polar2dial,pcolor_flux
max_val = 13
min_val = 7.5
    
for ii in range(0,100):
    results = model.predict( XX_test[0+ii*100:2+ii*100,:])

    fig= plt.figure(figsize=(12,10))
    ax1 = plt.subplot2grid((4,2), (0,0), colspan=2,rowspan=1)
    ax3 = plt.subplot2grid((4,2), (1,0), colspan=2,rowspan=1)
    ax4 = plt.subplot2grid((4,2), (2,0), rowspan=2,colspan=1,polar=True)
    ax5 = plt.subplot2grid((4,2), (2, 1), rowspan=2,colspan=1,polar=True)




    ax1.plot(-df_val['AL'], marker=mark[1],markersize=3)
    ax1.plot(df_val['AE'], marker=mark[2],markersize=3)
    ax1.plot(df_val['AU'], marker=mark[3],markersize=3)
    ax1.plot([df_val.index[ii*100],df_val.index[ii*100]],[range1,range2],'--')

    ax1.legend(['-AL','AE','AU'])
    ax1.set_xlabel('time (month, day, year labels)')
    ax1.set_title('Example Magnetic Indices',fontweight="bold")

Traceback (most recent call last):
    mlatgridN = np.linspace(90,45,num=128)
    mltgridN =  np.linspace(0,24,num=128)   
    flux = 10**(results[0,:,:,0])*1.60218e-12*3.14159


    pcolor_kwargs = {'cmap':'gnuplot','vmin':(10**7.5)*1.60218e-12*3.14159,'vmax':(10**12.5)*1.60218e-12*3.14159}
    mappableN = pcolor_flux(ax5,mlatgridN,mltgridN,flux,'N',**pcolor_kwargs)

    ax5.set_title('Predicted Electron Precipitation Energy Flux (Neural Net)',fontweight="bold", fontsize='medium',pad=10)


    ax5.set_theta_zero_location('S')
    theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
    theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
    ax5.set_thetagrids(theta_label_values,labels=theta_labels)

    r_label_values = 90.-np.array([80.,70.,60.,50.])
    r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
    ax5.set_rgrids(r_label_values,labels=r_labels)
    ax5.set_rlim([0.,45.])


    ###########################3
    # Ovation
    ########################
    dt = y_val_log.index[ii*100]
    print(dt)
    auroral_types = ['diff','mono','wave','ions']
    # axS = f.add_subplot(122,projection='polar')
    for jj in range(0,3):
        atype = auroral_types[jj]
        jtype ="energy"
        bincorrect = True
        combine_hemispheres = True
        dtstr = dt.strftime('%Y%m%d %H:%M')
        if jtype=='average energy':
            estimator = AverageEnergyEstimator(atype)
            get_precip_for_time = estimator.get_eavg_for_time
        else:
            estimator = FluxEstimator(atype,jtype)
            get_precip_for_time = estimator.get_flux_for_time


        tflux_kwargs = {'combine_hemispheres':combine_hemispheres,
                        'return_dF':True}
        fluxtupleN = get_precip_for_time(dt,hemi='N',**tflux_kwargs)
        mlatgridN,mltgridN,fluxgridN,newell_coupling = fluxtupleN
        fluxtupleS = get_precip_for_time(dt,hemi='S',**tflux_kwargs)
        mlatgridS,mltgridS,fluxgridS,newell_coupling = fluxtupleS

        if bincorrect:
            bcN = BinCorrector(mlatgridN,mltgridN)
            fluxgridN = bcN.fix(fluxgridN)
            bcS = BinCorrector(mlatgridS,mltgridS)
            fluxgridS = bcS.fix(fluxgridS)
            print("Correction Applied")

        if jj== 0:
            fluxgridN_sum = fluxgridN
            fluxgridS_sum = fluxgridS
        else:
            fluxgridN_sum = fluxgridN_sum+fluxgridN
            fluxgridS_sum = fluxgridN_sum+fluxgridS

    mappableN = pcolor_flux(ax4,mlatgridN,mltgridN,fluxgridN_sum,'N',**pcolor_kwargs)

    ax4.set_title('Predicted Electron Precipitation Energy Flux (OVATION Pyme)',pad =10,fontweight="bold", fontsize='medium')


    ax4.set_theta_zero_location('S')
    theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
    theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
    ax4.set_thetagrids(theta_label_values,labels=theta_labels,fontsize='medium', )

    r_label_values = 90.-np.array([80.,70.,60.,50.])
    r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
    ax4.set_rgrids(r_label_values,labels=r_labels)
    ax4.set_rlim([0.,45.])
    
    ax4.scatter(np.ones((20))*23/24*2*3.14159 ,                   np.linspace(0,40,20)      )     
    ax5.scatter(np.ones((20))*23/24*2*3.14159 ,                   np.linspace(0,45,20 )     )     

    plt.colorbar(mappableN,ax=ax5,label='Total Energy Flux [erg/cm^s/s]')
    
    ax3.set_title('Log Scale Electron Precipitation Energy Flux [Log10 eV/cm^2/s]'+' '+str(dt),fontweight="bold")
    ax3.scatter(np.linspace(90,50,80),np.log10(fluxgridN_sum[:,int(23/24*96)]/1.60218e-12), marker=mark[2])
    ax3.scatter(np.linspace(90,45,128),np.log10(flux[:,int(23/24*128)]/1.60218e-12), marker=mark[3])
    ax3.legend(['OVATION Pyme model','neural net: 0.5 deg / Mlat'])
    ax3.set_ylim(top=max_val,bottom=min_val)
    fig.tight_layout() 

    
    plt.show()

In [ ]:
mark= ['s', 'o', 'D', 'v']

%matplotlib inline
plt.ioff()
range1=np.min(np.min([-df_val['AL'].values,df_val['AE'].values,df_val['AU'].values]))
range2=np.max(np.max([-df_val['AL'].values,df_val['AE'].values,df_val['AU'].values]))
os.system('source /home/jackalak/heartbeat/cdf38_0-dist/bin/definitions.B')
os.environ["CDF_LIB"] = '/home/jackalak/heartbeat/cdf38_0-dist/lib'
from spacepy import pycdf

from ovationpyme.ovation_prime import FluxEstimator,AverageEnergyEstimator,BinCorrector
from ovationpyme.ovation_utilities import calc_avg_solarwind
from ovationpyme.ovation_plotting import latlt2polar,polar2dial,pcolor_flux
max_val = 13
min_val = 7.5
    
for ii in range(0,100):
    results = model.predict( XX_test[0+ii*100:2+ii*100,:])

    fig= plt.figure(figsize=(12,10))
    ax1 = plt.subplot2grid((4,2), (0,0), colspan=2,rowspan=1)
    ax3 = plt.subplot2grid((4,2), (1,0), colspan=2,rowspan=1)
    ax4 = plt.subplot2grid((4,2), (2,0), rowspan=2,colspan=1,polar=True)
    ax5 = plt.subplot2grid((4,2), (2, 1), rowspan=2,colspan=1,polar=True)




    ax1.plot(-df_val['AL'], marker=mark[1],markersize=3)
    ax1.plot(df_val['AE'], marker=mark[2],markersize=3)
    ax1.plot(df_val['AU'], marker=mark[3],markersize=3)
    ax1.plot([df_val.index[ii*100],df_val.index[ii*100]],[range1,range2],'--')

    ax1.legend(['-AL','AE','AU'])
    ax1.set_xlabel('time (month, day, year labels)')
    ax1.set_title('Example Magnetic Indices',fontweight="bold")

    mlatgridN = np.linspace(90,45,num=128)
    mltgridN =  np.linspace(0,24,num=128)   
    flux = 10**(results[0,:,:,0])*1.60218e-12*3.14159


    pcolor_kwargs = {'cmap':'gnuplot','vmin':7.5,'vmax':12.5}
    mappableN = pcolor_flux(ax5,mlatgridN,mltgridN,np.log10(flux/1.60218e-12),'N',**pcolor_kwargs)

    ax5.set_title('Log10 Predicted Electron Precipitation Energy Flux (Neural Net)',fontweight="bold", fontsize='medium',pad=10)


    ax5.set_theta_zero_location('S')
    theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
    theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
    ax5.set_thetagrids(theta_label_values,labels=theta_labels)

    r_label_values = 90.-np.array([80.,70.,60.,50.])
    r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
    ax5.set_rgrids(r_label_values,labels=r_labels)
    ax5.set_rlim([0.,45.])


    ###########################3
    # Ovation128
    ########################
    dt = y_val_log.index[ii*100]
    print(dt)
    auroral_types = ['diff','mono','wave','ions']
    # axS = f.add_subplot(122,projection='polar')
    for jj in range(0,3):
        atype = auroral_types[jj]
        jtype ="energy"
        bincorrect = True
        combine_hemispheres = True
        dtstr = dt.strftime('%Y%m%d %H:%M')
        if jtype=='average energy':
            estimator = AverageEnergyEstimator(atype)
            get_precip_for_time = estimator.get_eavg_for_time
        else:
            estimator = FluxEstimator(atype,jtype)
            get_precip_for_time = estimator.get_flux_for_time


        tflux_kwargs = {'combine_hemispheres':combine_hemispheres,
                        'return_dF':True}
        fluxtupleN = get_precip_for_time(dt,hemi='N',**tflux_kwargs)
        mlatgridN,mltgridN,fluxgridN,newell_coupling = fluxtupleN
        fluxtupleS = get_precip_for_time(dt,hemi='S',**tflux_kwargs)
        mlatgridS,mltgridS,fluxgridS,newell_coupling = fluxtupleS

        if bincorrect:
            bcN = BinCorrector(mlatgridN,mltgridN)
            fluxgridN = bcN.fix(fluxgridN)
            bcS = BinCorrector(mlatgridS,mltgridS)
            fluxgridS = bcS.fix(fluxgridS)
            print("Correction Applied")

        if jj== 0:
            fluxgridN_sum = fluxgridN
            fluxgridS_sum = fluxgridS
        else:
            fluxgridN_sum = fluxgridN_sum+fluxgridN
            fluxgridS_sum = fluxgridN_sum+fluxgridS

    mappableN = pcolor_flux(ax4,mlatgridN,mltgridN,np.log10(fluxgridN_sum/1.60218e-12),'N',**pcolor_kwargs)

    ax4.set_title('Log10 Predicted Electron Precipitation Energy Flux (OVATION Pyme)',pad =10,fontweight="bold", fontsize='medium')


    ax4.set_theta_zero_location('S')
    theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
    theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
    ax4.set_thetagrids(theta_label_values,labels=theta_labels,fontsize='medium', )

    r_label_values = 90.-np.array([80.,70.,60.,50.])
    r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
    ax4.set_rgrids(r_label_values,labels=r_labels)
    ax4.set_rlim([0.,45.])
    
    ax4.scatter(np.ones((20))*23/24*2*3.14159 ,                   np.linspace(0,40,20)      )     
    ax5.scatter(np.ones((20))*23/24*2*3.14159 ,                   np.linspace(0,45,20 )     )     

    plt.colorbar(mappableN,ax=ax5,label='Total Energy Flux [erg/cm^s/s]')
    
    ax3.set_title('Log Scale Electron Precipitation Energy Flux [Log10 eV/cm^2/s]'+' '+str(dt),fontweight="bold")
    ax3.scatter(np.linspace(90,50,80),np.log10(fluxgridN_sum[:,int(23/24*96)]/1.60218e-12), marker=mark[2])
    ax3.scatter(np.linspace(90,45,128),np.log10(flux[:,int(23/24*128)]/1.60218e-12), marker=mark[3])
    ax3.legend(['OVATION Pyme model','neural net: 0.5 deg / Mlat'])
    ax3.set_ylim(top=max_val,bottom=min_val)
    fig.tight_layout() 

    
    plt.show()

In [ ]:
def plot_hemisphere_CSV_val(scaler_X, model, features, test, XX_test):

    mark= ['s', 'o', 'D', 'v']

    y_val_log = np.log10(test['ELE_TOTAL_ENERGY_FLUX']+.0001)

    num = test.shape[0]
    
    %matplotlib inline
    plt.ioff()
    ovation_val = []
    result_val = []
    
    range1=np.max(np.max([-test['AL'].values,test['AE'].values,test['AU'].values]))
    
    ml_hemi = []
    ovation_hemi = []
    
    for i in range(0,num):


       #################################
        #
        #ML model
        ##########################
        results = model.predict( XX_test[i:i+1,:])
        mlatgridN = np.linspace(90,45,num=128)
        mltgridN =  np.linspace(0,24,num=128)   
        flux = 10**(results[0,:,:,0])*1.60218e-12*3.14159




        ###########################3
        # Ovation
        ########################
        dt = test.index[i]
        auroral_types = ['diff','mono','wave','ions']
        for jj in range(0,3):
            atype = auroral_types[jj]
            jtype ="energy"
            bincorrect = True
            combine_hemispheres = True
            dtstr = dt.strftime('%Y%m%d %H:%M')
            if jtype=='average energy':
                estimator = AverageEnergyEstimator(atype)
                get_precip_for_time = estimator.get_eavg_for_time
            else:
                estimator = FluxEstimator(atype,jtype)
                get_precip_for_time = estimator.get_flux_for_time


            tflux_kwargs = {'combine_hemispheres':combine_hemispheres,
                            'return_dF':True}
            fluxtupleN = get_precip_for_time(dt,hemi='N',**tflux_kwargs)
            mlatgridN,mltgridN,fluxgridN,newell_coupling = fluxtupleN
            fluxtupleS = get_precip_for_time(dt,hemi='S',**tflux_kwargs)
            mlatgridS,mltgridS,fluxgridS,newell_coupling = fluxtupleS

            if bincorrect:
                bcN = BinCorrector(mlatgridN,mltgridN)
                fluxgridN = bcN.fix(fluxgridN)
                bcS = BinCorrector(mlatgridS,mltgridS)
                fluxgridS = bcS.fix(fluxgridS)
                print("Correction Applied")

            if jj== 0:
                fluxgridN_sum = fluxgridN
                fluxgridS_sum = fluxgridS
            else:
                fluxgridN_sum = fluxgridN_sum+fluxgridN
                fluxgridS_sum = fluxgridN_sum+fluxgridS

           
       
        pt = i

        result_val.append( 
            flux[int((90-test['SC_AACGM_LAT'][pt])/45*128),int((test['SC_AACGM_LTIME'][pt])/24*128)] )
        if  test['SC_AACGM_LAT'][pt] <= 50:
            ovation_val.append( fluxgridN_sum[0,int((test['SC_AACGM_LTIME'][pt])/24*96)] )
        else:
            ovation_val.append( 
                fluxgridN_sum[int((90-test['SC_AACGM_LAT'][pt])/40*80),int((test['SC_AACGM_LTIME'][pt])/24*96)] )

       
        ml_hemi.append(flux)
        ovation_hemi.append(fluxgridN_sum)
    
        
    plt.show()
    
    for i in range(0,num):
        
        flux = ml_hemi[i]
        fluxgridN_sum =ovation_hemi[i]

        fig= plt.figure(figsize=(20,12))
        ax3 = plt.subplot2grid((4,4), (0,0), colspan=4,rowspan=1)
        ax1 = plt.subplot2grid((4,4), (1,0), colspan=4,rowspan=1)
        ax4 = plt.subplot2grid((4,4), (2,0), rowspan=2,colspan=2,polar=True)
        ax5 = plt.subplot2grid((4,4), (2, 2), rowspan=2,colspan=2,polar=True)
        
        ax1.plot(-test['AL'], marker=mark[1],markersize=3)
        ax1.plot(test['AE'], marker=mark[2],markersize=3)
        ax1.plot(test['AU'], marker=mark[3],markersize=3)
        ax1.plot([test.index[i],test.index[i]],[0,10000],'--')

        ax1.legend(['-AL','AE','AU'])
        ax1.set_ylim(top=range1)
        ax1.set_xlim(right=test.index[-1])
        ax1.set_xlabel('time (month, day, year labels)')
        ax1.set_title('Example Magnetic Indices',fontweight="bold")

       #################################
        #
        #ML model
        ##########################
        mlatgridN = np.linspace(90,45,num=128)
        mltgridN =  np.linspace(0,24,num=128)  



        pcolor_kwargs = {'cmap':'gnuplot','vmin':(10**0)*1.60218e-12,'vmax':(10**12.5)*1.60218e-12*3.14159}
        mappableN = pcolor_flux(ax5,mlatgridN,mltgridN,flux,'N',**pcolor_kwargs)

        ax5.set_title('Predicted Electron Precipitation Energy Flux (Neural Net)',fontweight="bold", fontsize='medium',pad=10)
        ax5.set_theta_zero_location('S')
        theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
        theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
        ax5.set_thetagrids(theta_label_values,labels=theta_labels)

        r_label_values = 90.-np.array([80.,70.,60.,50.])
        r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
        ax5.set_rgrids(r_label_values,labels=r_labels)
        ax5.set_rlim([0.,45.])


        ###########################3
        # Ovation
        ########################
        dt = test.index[i]
        dtstr = dt.strftime('%Y%m%d %H:%M')
        mlatgridN = np.linspace(90,50,num=80)
        mltgridN = np.linspace(90,45,num=96)

        mappableN = pcolor_flux(ax4,mlatgridN,mltgridN,fluxgridN_sum,'N',**pcolor_kwargs)

        ax4.set_title('Predicted Electron Precipitation Energy Flux (OVATION Pyme)',pad =10,fontweight="bold", fontsize='medium')
        ax4.set_theta_zero_location('S')
        theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
        theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
        ax4.set_thetagrids(theta_label_values,labels=theta_labels,fontsize='medium', )

        r_label_values = 90.-np.array([80.,70.,60.,50.])
        r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
        ax4.set_rgrids(r_label_values,labels=r_labels)
        ax4.set_rlim([0.,45.])
        plt.colorbar(mappableN,ax=ax5,label='Total Flux [erg/cm^2/s]',fraction=0.05,pad=0.09)
        plt.colorbar(mappableN,ax=ax4,label='Total Flux [erg/cm^2/s]',fraction=0.05,pad=0.09)

           
        ax4.scatter(test['SC_AACGM_LTIME'][i]/24*2*3.14159,90-test['SC_AACGM_LAT'][i])     
        ax5.scatter(test['SC_AACGM_LTIME'][i]/24*2*3.14159,90-test['SC_AACGM_LAT'][i])     
        ax4.tick_params(axis='y', colors='white')
        ax5.tick_params(axis='y', colors='white')


        ax3.set_title('Log Scale Electron Precipitation Energy Flux [Log10 eV/cm^2/s]'+' '+str(dt),fontweight="bold")
        ax3.set_title('Log Scale Electron Precipitation Energy Flux Log10 [eV/cm^2/s]'+' '+str(dt),fontweight="bold")
        ax3.plot(test.index,np.log10(np.array(ovation_val)/1.60218e-12), marker=mark[1])
        ax3.plot(test.index,np.log10(np.array(result_val)/1.60218e-12), marker=mark[2])
        ax3.plot(test.index,np.log10(10**(y_val_log)*3.14159), marker=mark[3])
        ax3.plot([test.index[i],test.index[i]],[7.5,13.5],'--')
        ax3.legend(['OVATION Pyme model','neural net: 0.5 deg / Mlat','measured value','current time'])
        ax3.set_ylim(top=13.5,bottom=7.5)
        ax3.set_xlim(right=test.index[-1])

        fig.tight_layout() 
        name = 'figures/movie_val_'+str(i) + '.png'
        fig.savefig(name,dpi=400)
        
    plt.show()
    
    #os.system('ffmpeg -r:v 5 -i "figures/movie_val_%01d.png" -codec:v libx264 -preset veryslow  "movie_val_2013_SC17.mp4";ffmpeg -i movie_val_2013_SC17.mp4 -c:v libx264 -c:a libmp3lame -b:a 384K movie_val_2013_SC17.avi;')
    return result_val, ovation_val


In [ ]:
features = ['SC_AACGM_LAT', 'ELE_TOTAL_ENERGY_FLUX', 'ID_SC', 'sin_ut',
                             'cos_ut', 'sin_doy', 'cos_doy', 'sin_SC_AACGM_LTIME', 'cos_SC_AACGM_LTIME',
                             'F107', 'AE', 'AL', 'AU', 'SymH', 
                             'F107_6hr', 'AE_6hr', 'AL_6hr', 'AU_6hr', 'SymH_6hr', 'F107_5hr', 'AE_5hr', 'AL_5hr', 'AU_5hr', 'SymH_5hr', 
                             'F107_3hr', 'AE_3hr', 'AL_3hr', 'AU_3hr', 'SymH_3hr', 
                             'F107_1hr', 'AE_1hr', 'AL_1hr', 'AU_1hr', 'SymH_1hr', 
                             ]

result_val, ovation_val = plot_hemisphere_CSV_val(scaler_X, model, features, df_val[0:df_val.shape[0]:600], XX_test[0:df_val.shape[0]:600,:])


In [ ]:
np.average( (y_val_log[0:df_val.shape[0]:600]-np.log10(np.array(result_val)/1.60218e-12/3.14159))**2)


In [ ]:
def for_CSV_val(scaler_X, model, features, test, XX_test):

    mark= ['s', 'o', 'D', 'v']

    y_val_log = np.log10(test['ELE_TOTAL_ENERGY_FLUX']+.0001)

    num = test.shape[0]
    
    %matplotlib inline
    plt.ioff()
    result_val = []

    
    for i in range(0,num):


       #################################
        #
        #ML model
        ##########################
        results = model.predict( XX_test[i:i+1,:])
        mlatgridN = np.linspace(90,45,num=128)
        mltgridN =  np.linspace(0,24,num=128)   
        flux = results[0,:,:,0]




           
       
        pt = i

        result_val.append( 
            flux[int((90-test['SC_AACGM_LAT'][pt])/45*128),int((test['SC_AACGM_LTIME'][pt])/24*128)] )

       
    
        

    return result_val

In [ ]:
result_val = for_CSV_val(scaler_X, model, features, df_val, XX_test)
np.average((y_val_log-(np.array(result_val)))**2)


In [ ]:
plt.figure()
plt.hist(y_val_log,  alpha=0.5, bins=200)
plt.hist( result_val, alpha=0.5, bins=200)
plt.show()

In [ ]:
dpi=200
df_results = pd.DataFrame(data=result_val, index = df_val.index)

plot_model(df_results, df_val)

In [ ]:
df_val